In [95]:
from datetime import date
import numpy as np
import open3d as o3d

today = date.today()
#Normals Vis
import open3d as o3d
from pyntcloud import PyntCloud

pcd = o3d.io.read_point_cloud("/home/sashawald/Documents/shaving_code-main/clouds/"+str(today)+".ply")

#Visualize Original
#print(pcd)
#o3d.visualization.draw_geometries([pcd])
R = np.identity(3)  
extent = np.ones(3)/.31 # trying to create a bounding box below 1 unit
center = np.zeros(3) 
obb = o3d.geometry.OrientedBoundingBox(center,R,extent)
pcd = pcd.crop(obb)
dpcd = pcd.voxel_down_sample(voxel_size=0.035)
#use draw_geometries if you don't need to edit the result
o3d.visualization.draw_geometries_with_editing([dpcd])
#cloud = PyntCloud.from_instance("open3d", pcd)
#scene = cloud.plot(return_scene=True, position=(1,1,1))


In [101]:
#Code  for picking specific points: shift+left click to select, shift+right click to deselect
#close visualization window to return selected points
import numpy as np
from LineMesh import *

vis = o3d.visualization.VisualizerWithEditing()
vis.create_window()
vis.add_geometry(dpcd)
vis.run()  
vis.destroy_window()
pdata = vis.get_picked_points()


[Open3D INFO] Picked point #1114 (-0.044, -0.18, -1.4) to add in queue.
[Open3D INFO] Picked point #1982 (0.34, -0.16, -1.5) to add in queue.


In [30]:
alpha = 0.25
mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_alpha_shape(dpcd,alpha)
mesh.compute_vertex_normals()
o3d.visualization.draw_geometries([mesh],mesh_show_back_face=True)

In [11]:
#Poisson (not using for now)
print('run Poisson surface reconstruction')
with o3d.utility.VerbosityContextManager(
        o3d.utility.VerbosityLevel.Debug) as cm:
    mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(
        pcd, depth=9)
print(mesh)
o3d.visualization.draw_geometries([mesh],
                                  zoom=0.664,
                                  front=[-0.4761, -0.4698, -0.7434],
                                  lookat=[1.8900, 3.2596, 0.9284],
                                  up=[0.2304, -0.8825, 0.4101])

run Poisson surface reconstruction
[Open3D DEBUG] Input Points / Samples: 303839 / 124031
[Open3D DEBUG] #   Got kernel density: 0.0335779 (s), 650.297 (MB) / 650.297 (MB) / 650 (MB)
[Open3D DEBUG] #     Got normal field: 0.351765 (s), 669.391 (MB) / 669.391 (MB) / 669 (MB)
[Open3D DEBUG] Point weight / Estimated Area: 3.253682e-06 / 9.885955e-01
[Open3D DEBUG] #       Finalized tree: 0.257862 (s), 710.621 (MB) / 710.621 (MB) / 710 (MB)
[Open3D DEBUG] #  Set FEM constraints: 0.129563 (s), 710.621 (MB) / 710.621 (MB) / 710 (MB)
[Open3D DEBUG] #Set point constraints: 0.0465119 (s), 710.621 (MB) / 710.621 (MB) / 710 (MB)
[Open3D DEBUG] Leaf Nodes / Active Nodes / Ghost Nodes: 1184317 / 1329112 / 24393
[Open3D DEBUG] Memory Usage: 710.621 MB
[Open3D DEBUG] # Linear system solved: 1.20031 (s), 746.023 (MB) / 746.023 (MB) / 746 (MB)
[Open3D DEBUG] Got average: 0.00598001 (s), 746.023 (MB) / 746.023 (MB) / 746 (MB)
[Open3D DEBUG] Iso-Value: 4.981425e-01 = 1.513551e+05 / 3.038390e+05
[Open3D D

In [110]:
import dijkstra as djk
graph_d = djk.Graph()
#found that it doesn't work well with a mesh - will have to revisit
""" for edge in edgeindices:
    p1 = vertices[edge[0]]
    p2 = vertices[edge[1]]
    dist = np.linalg.norm(p1 - p2)
    graph_d.add_edge(edge[0],edge[1],dist)
 """
pcd_tree = o3d.geometry.KDTreeFlann(dpcd)
count=0
for v in dpcd.points:
    [k, idx, _] = pcd_tree.search_knn_vector_3d(dpcd.points[count], 5)
    for v_curr in idx:
        if not np.array_equal(v,v_curr):
            p1 = v
            p2 = v_curr
            sq_d = np.sum((p1-p2)**2, axis=0)
            dist=np.sqrt(sq_d)
            graph_d.add_edge(count,v_curr,dist)
    count=count+1

In [111]:
dijkstra = djk.DijkstraSPF(graph_d,pdata[0])
foundpcd = dpcd.select_by_index(dijkstra.get_path((pdata[1])))
foundpcd.paint_uniform_color([1, 0, 0])
dpcd.paint_uniform_color([0,1,0])
o3d.visualization.draw_geometries([dpcd,foundpcd])